# Kaggle house_price_XGBoost

## ライブラリのインポート

In [1]:
# # ライブラリのインポート
import pandas as pd
import numpy as np
from numpy import nan
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
%matplotlib inline

## データの読み込み

In [2]:
# # trainデータ
train = pd.read_csv('train.csv', usecols=["Id","SalePrice","GarageYrBlt","LotFrontage","BsmtCond","PoolQC","TotalBsmtSF","Condition1","FireplaceQu","YrSold","HouseStyle","1stFlrSF","2ndFlrSF","MSSubClass","MSZoning","GarageCars","OverallQual"])
# # testデータ
test = pd.read_csv('test.csv', usecols=["Id","GarageYrBlt","LotFrontage","BsmtCond","PoolQC","TotalBsmtSF","Condition1","FireplaceQu","YrSold","HouseStyle","1stFlrSF","2ndFlrSF","MSSubClass","MSZoning","GarageCars","OverallQual"])
# # サンプル提出ファイル
sample_file = pd.read_csv('sample_submission.csv')

In [3]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,Condition1,HouseStyle,OverallQual,BsmtCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,FireplaceQu,GarageYrBlt,GarageCars,PoolQC,YrSold,SalePrice
0,1,60,RL,65.0,Norm,2Story,7,TA,856,856,854,NaN,2003.0,2,NaN,2008,208500
1,2,20,RL,80.0,Feedr,1Story,6,TA,1262,1262,0,TA,1976.0,2,NaN,2007,181500
2,3,60,RL,68.0,Norm,2Story,7,TA,920,920,866,TA,2001.0,2,NaN,2008,223500
3,4,70,RL,60.0,Norm,2Story,7,Gd,756,961,756,Gd,1998.0,3,NaN,2006,140000
4,5,60,RL,84.0,Norm,2Story,8,TA,1145,1145,1053,TA,2000.0,3,NaN,2008,250000


In [4]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,Condition1,HouseStyle,OverallQual,BsmtCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,FireplaceQu,GarageYrBlt,GarageCars,PoolQC,YrSold
0,1461,20,RH,80.0,Feedr,1Story,5,TA,882.0,896,0,NaN,1961.0,1.0,NaN,2010
1,1462,20,RL,81.0,Norm,1Story,6,TA,1329.0,1329,0,NaN,1958.0,1.0,NaN,2010
2,1463,60,RL,74.0,Norm,2Story,5,TA,928.0,928,701,TA,1997.0,2.0,NaN,2010
3,1464,60,RL,78.0,Norm,2Story,6,TA,926.0,926,678,Gd,1998.0,2.0,NaN,2010
4,1465,120,RL,43.0,Norm,1Story,8,TA,1280.0,1280,0,NaN,1992.0,2.0,NaN,2010


詳細はサイトへ https://www.kaggle.com/c/house-prices-advanced-regression-techniques

## 前処理

In [7]:
# # 欠損値が多いもの,相関が無さそうなものを削除

train = train.drop('Id', axis=1)
# test = test.drop('Id', axis=1)


# # 欠損値を補完

train = train.fillna(train['LotFrontage'].median())
test = test.fillna(test['LotFrontage'].median())
train = train.fillna(train['GarageYrBlt'].median())
test = test.fillna(test['GarageYrBlt'].median())
train = train.fillna(train['TotalBsmtSF'].median())
test = test.fillna(test['TotalBsmtSF'].median())
train = train.fillna(train['GarageCars'].median())
test = test.fillna(test['GarageCars'].median())

# # マッピング

train = pd.get_dummies(train, drop_first=True, dummy_na=True, 
               columns=['MSSubClass', 'FireplaceQu', 'BsmtCond', "PoolQC", 'MSZoning', 'Condition1'
                       , 'HouseStyle', 'YrSold'])
test = pd.get_dummies(test, drop_first=True, dummy_na=True, 
               columns=['MSSubClass', 'FireplaceQu', 'BsmtCond', "PoolQC", 'MSZoning', 'Condition1'
                       , 'HouseStyle', 'YrSold'])
# # それ以外
train = train.drop('MSSubClass_40.0', axis=1)

In [9]:
train['LotFrontage'].fillna(train['LotFrontage'].median())

0       65.0
1       80.0
2       68.0
3       60.0
4       84.0
        ... 
1455    62.0
1456    85.0
1457    66.0
1458    68.0
1459    75.0
Name: LotFrontage, Length: 1460, dtype: float64

## 標準化・分割

In [10]:
# # X, yにデータを代入
# # ここで説明変数を適宜変更
X = train.values
y = train.loc[:, ['SalePrice']].values

# # データの標準化(標準正規分布)
scaler = StandardScaler()
X_std = scaler.fit_transform(X)
# y_std = scaler.fit_transform(y)

# # ホールド・アウト法による分割
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.3, random_state=0)

## 学習

In [13]:
import xgboost as xgb

regr = xgb.XGBRegressor(
                 colsample_bytree=0.2,
                 gamma=0.0,
                 learning_rate=0.01,
                 max_depth=4,
                 min_child_weight=1.5,
                 n_estimators=7200,                                                                  
                 reg_alpha=0.9,
                 reg_lambda=0.6,
                 subsample=0.2,
                 seed=42,
                 silent=1)

regr.fit(X_train, y_train)
y_predd = regr.predict(test_data)


[17:05:46] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




## 配布されたテストデータで予測・提出

In [12]:
# # testデータからデータの取り出し
test_data = test
test_data = test_data.drop('Id', axis=1)
test_data = test_data.values
test_data = scaler.fit_transform(test_data)

# # 提出データの予測
y_predd = lasso.predict(test_data)

In [14]:
# # 提出csvファイルの作成
submit_file = pd.DataFrame({'Id' : test['Id'], 'SalePrice' : y_predd})
submit_file.to_csv('submit.csv', index = False)